<a href="https://colab.research.google.com/github/zarawellsdevos/googlecolab/blob/main/one_legged_cycling_time_trial_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# One-Legged Cycling Time Trial

Here we develop and solve an optimal control problem to find out:

*How to activate muscles in the leg to minimize the time taken to travel a specified distance from a standstill on a bicycle?*

We simplify the system by modeling a single human leg with four lumped muscular-tendons acuating the knee and ankle:

- An extensor that wraps over the knee
- A flexor under the thigh to flex the knee
- Dorsiflexor at the ankle joint
- Plantar flexor at the ankle joint

The reaction force at the foot-pedal interface is modeled to mimic the torque felt if accelerating the whole bicycle with rider on flat ground.

A schematic below shows the elements of the model.

The goal will be travel a specific distance in the shortest amount of time given that the leg muscles have to coordinate and can only be excited from to contract.

Changing the various parameters of the model can decrease or increase the time it takes to cross the finish line.

![](https://github.com/mechmotum/one-legged-time-trial/blob/main/one-legged-time-trial.svg?raw=1)

## Setting up the coding environment

These code blocks will make sure all the needed packages are installed on Colab.

In [35]:
import os
if 'google.colab' in str(get_ipython()):
    !pip install -q condacolab
    import condacolab

    try:
        condacolab.check()
    except AssertionError:
        print("Condacolab not yet installed, installing now. Session will crash and restart")
        condacolab.install()
    except:
        print("Something went wrong")

    if os.path.isfile("one-legged-cycling-time-trial-env.yml"):
        print("Environment already setup, proceed.")
    else:
        !wget https://raw.githubusercontent.com/mechmotum/one-legged-time-trial/main/one-legged-cycling-time-trial-env.yml

    try:
        import opty
    except ImportError:
         print("Environment will be installed. This can take a few minutes.")
         !conda env update -n base -f one-legged-cycling-time-trial-env.yml

✨🍰✨ Everything looks OK!
Environment already setup, proceed.


In [36]:
from opty.direct_collocation import Problem
from opty.utils import parse_free
from scipy.optimize import fsolve
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import sympy as sm
import sympy.physics.biomechanics as bm
import sympy.physics.mechanics as me
from IPython.display import HTML

In [37]:
me.init_vprinting()

# Define coordinates and variables

## Coordinates

$q_1 =$ crank angle \
$q_2 =$ pedal angle \
$q_3 =$ ankle angle \
$q_4 =$ knee angle \
$u_1 =$ crank angular rate (cadence) \
$u_2 =$ pedal angular rate \
$u_3 =$ ankle angular rate \
$u_4 =$ knee angular rate

In [38]:
t = me.dynamicsymbols._t
q1, q2, q3, q4 = me.dynamicsymbols("q1, q2, q3, q4", real=True)
u1, u2, u3, u4 = me.dynamicsymbols("u1, u2, u3, u4", real=True)
q = sm.Matrix([q1, q2, q3, q4])
u = sm.Matrix([u1, u2, u3, u4])

q, u

⎛⎡q₁⎤  ⎡u₁⎤⎞
⎜⎢  ⎥  ⎢  ⎥⎟
⎜⎢q₂⎥  ⎢u₂⎥⎟
⎜⎢  ⎥, ⎢  ⎥⎟
⎜⎢q₃⎥  ⎢u₃⎥⎟
⎜⎢  ⎥  ⎢  ⎥⎟
⎝⎣q₄⎦  ⎣u₄⎦⎠

## Constants
$l_s = $ seat tube length \
$l_c = $ crank length \
$l_f = $ distance from pedal-foot contact to ankle \
$l_l = $ lower leg length \
$l_u = $ upper leg length \
$\lambda = $ seat tube angle \
$g = $ acceleration due to gravity \
$r_k = $ knee wrapping radius \
$m_A = $ mass of crank \
$m_B = $ mass of foot and pedal \
$m_C = $ mass of lower leg \
$m_D = $ mass of upper leg \
$I_{Axx} = $ moment of inertia of crank \
$I_{Bxx} = $ moment of inertia of foot and pedal \
$I_{Cxx} = $ moment of inertia of lower leg \
$I_{Dxx} = $ moment of inertia of upper leg \
$J = $ rotational moment of inertia of a bicycle wheel \
$m = $ mass of the bicycle and cyclist \
$r_w = $ wheel radius \
$G = $ gear ratio between crank and wheel \
$C_r = $ coefficient of rolling resistance \
$C_D = $ coefficient of drag \
$\rho = $ density of air \
$A_r = $ frontal area of bicycle and cyclist

In [39]:
ls, lc, lf, ll, lu = sm.symbols("ls, lc, lf, ll, lu", real=True, positive=True)
lam, g, rk = sm.symbols("lam, g, rk", real=True, nonnegative=True)
mA, mB, mC, mD = sm.symbols("mA, mB, mC, mD", nonnegative=True)
IAzz, IBzz, ICzz, IDzz = sm.symbols("IAzz, IBzz, ICzz, IDzz", nonnegative=True)
J, m, rw, G, Cr, CD, rho, Ar = sm.symbols("J, m, rw, G, Cr, CD, rho, Ar", nonnegative=True)

## Reference Frames

$N = $ inertial reference frame for leg dynamics \
$A = $ crank \
$B = $ foot \
$C = $ lower leg \
$D = $ upper leg

In [40]:
N, A, B, C, D = sm.symbols("N, A, B, C, D", cls=me.ReferenceFrame)

A.orient_axis(N, N.z, q1)  # crank angle
B.orient_axis(A, A.z, q2)  # pedal/foot angle
C.orient_axis(B, B.z, q3)  # ankle angle
D.orient_axis(C, C.z, q4)  # knee angle

A.set_ang_vel(N, u1 * N.z)
B.set_ang_vel(A, u2 * A.z)
C.set_ang_vel(B, u3 * B.z)
D.set_ang_vel(C, u4 * C.z)

# Point Kinematics
$P_1 =$ crank center \
$P_2 =$ pedal center \
$P_3 =$ ankle \
$P_4 =$ knee \
$P_5 =$ hip \
$P_6 =$ seat center \
$P_7 =$ heel \
$P_8 =$ knee muscle lower leg insertion point \
$P_9 =$ ankle muscle lower leg insertion point

In [41]:
P1, P2, P3, P4, P5, P6, P7, P8, P9 = sm.symbols(
    "P1, P2, P3, P4, P5, P6, P7, P8, P9", cls=me.Point
)
Ao, Bo, Co, Do = sm.symbols("Ao, Bo, Co, Do", cls=me.Point)

Ao.set_pos(P1, 0 * A.x)
P2.set_pos(P1, lc * A.x)  # pedal center
Bo.set_pos(P2, lf / 2 * B.x)  # foot mass center
P3.set_pos(P2, lf * B.x)  # ankle
P7.set_pos(P2, 3 * lf / 2 * B.x)  # heel
Co.set_pos(P3, ll / 2 * C.x)  # lower leg mass center
P4.set_pos(P3, ll * C.x)  # knee
Do.set_pos(P4, lu / 2 * D.x)  # upper leg mass center
P5.set_pos(P4, lu * D.x)  # hip
P6.set_pos(P1, -ls * sm.cos(lam) * N.x + ls * sm.sin(lam) * N.y)  # seat
P8.set_pos(P3, ll / 6 * C.x)
P9.set_pos(P4, -2 * rk * C.x)

P1.set_vel(N, 0)
P6.set_vel(N, 0)
Ao.v2pt_theory(P1, N, A)
P2.v2pt_theory(P1, N, A)
P7.v2pt_theory(P2, N, B)
Bo.v2pt_theory(P2, N, B)
P3.v2pt_theory(P2, N, B)
Co.v2pt_theory(P3, N, C)
P8.v2pt_theory(P3, N, C)
P9.v2pt_theory(P3, N, C)
P4.v2pt_theory(P3, N, C)
Do.v2pt_theory(P4, N, D)
P5.v2pt_theory(P4, N, D)

kindiff = sm.Matrix([ui - qi.diff() for ui, qi in zip(u, q)])

# Holonomic Constraints
The leg forms a kinematic loop and two holonomic constraints arise from this loop.

In [42]:
holonomic = (P5.pos_from(P1) - P6.pos_from(P1)).to_matrix(N)[:2, :]

qd_repl = {q1.diff(): u1, q2.diff(): u2, q3.diff(): u3, q4.diff(): u4}

mocon = me.msubs(holonomic.diff(t), qd_repl)

# Inertia

In [43]:
IA = me.Inertia.from_inertia_scalars(Ao, A, 0, 0, IAzz)
IB = me.Inertia.from_inertia_scalars(Bo, B, 0, 0, IBzz)
IC = me.Inertia.from_inertia_scalars(Co, C, 0, 0, ICzz)
ID = me.Inertia.from_inertia_scalars(Do, D, 0, 0, IDzz)

crank = me.RigidBody("crank", masscenter=Ao, frame=A, mass=mA, inertia=IA)
foot = me.RigidBody("foot", masscenter=Bo, frame=B, mass=mB, inertia=IB)
lower_leg = me.RigidBody("lower leg", masscenter=Co, frame=C, mass=mC, inertia=IC)
upper_leg = me.RigidBody("upper leg", masscenter=Do, frame=D, mass=mD, inertia=ID)

# Forces

In [44]:
gravB = me.Force(Bo, -mB * g * N.y)
gravC = me.Force(Co, -mC * g * N.y)
gravD = me.Force(Do, -mD * g * N.y)

# Crank Resistance

We model the resistance torque at the crank to be that which you would feel when accelerating the bicycle and cyclist along flat ground. The basic equations of motion of a point mass model of a cyclist are:

$$
(2J + m r_w^2)\dot{\omega} = C_r m g r_w + \operatorname{sgn}(-\omega) \frac{1}{2} \rho C_D A_r (-\omega r_w)^2 +T_w
$$

The angular speed of the rear wheel is related to the crank cadence by the gear ratio $G$:

$$
\omega = G u_1,  \dot{\omega} = G \dot{u}_1\\
$$

$$
G T_w = T_c \\
$$

$$
T_c = (2J + m r_w^2)G^2\dot{u}_1 - C_r m g r_w G - \operatorname{sgn}(-u_1) \frac{1}{2} \rho C_D A_r G^3 (-u_1 r_w)^2
$$

We will only allow forward motion, i.e. $u_1 < 0$, so the above equation reduces to:

$$
-T_c = -(2J + m r_w^2)G^2\dot{u}_1 + C_r m g r_w G + \frac{1}{2} \rho C_D A_r G^3 (u_1 r_w)^2
$$

In [45]:
# NOTE : we enforce later that u1 < 0 (forward pedaling), thus the
# resistance should be a posistive torque to resist the negative speed
# NOTE : using sm.sign() will break the constraint Jacobian due taking the
# derivative of sm.sign().

resistance = me.Torque(
    crank,
    (
        -(2 * J + m * rw**2) * G**2 * u1.diff()
        + Cr * m * g * rw * G
        + rho * CD * Ar * G**3 * rw**3 * u1**2 / 2
    )
    * N.z,
)

# Muscles
We lump all the muscles that contribute to joint torques at the knee and ankle into four simplified muscles. The quadriceps wrap over the knee. The other muscles act on linear pathways.

In [46]:
class ExtensorPathway(me.PathwayBase):
    def __init__(
        self,
        origin,
        insertion,
        axis_point,
        axis,
        parent_axis,
        child_axis,
        radius,
        coordinate,
    ):
        """A custom pathway that wraps a circular arc around a pin joint.
        This is intended to be used for extensor muscles. For example, a
        triceps wrapping around the elbow joint to extend the upper arm at
        the elbow.

        Parameters
        ==========
        origin : Point
            Muscle origin point fixed on the parent body (A).
        insertion : Point
            Muscle insertion point fixed on the child body (B).
        axis_point : Point
            Pin joint location fixed in both the parent and child.
        axis : Vector
            Pin joint rotation axis.
        parent_axis : Vector
            Axis fixed in the parent frame (A) that is directed from the pin
            joint point to the muscle origin point.
        child_axis : Vector
            Axis fixed in the child frame (B) that is directed from the pin
            joint point to the muscle insertion point.
        radius : sympyfiable
            Radius of the arc that the muscle wraps around.
        coordinate : sympfiable function of time
            Joint angle, zero when parent and child frames align. Positive
            rotation about the pin joint axis, B with respect to A.

        Notes
        =====
        Only valid for coordinate >= 0.

        """
        super().__init__(origin, insertion)
        self.origin = origin
        self.insertion = insertion
        self.axis_point = axis_point
        self.axis = axis.normalize()
        self.parent_axis = parent_axis.normalize()
        self.child_axis = child_axis.normalize()
        self.radius = radius
        self.coordinate = coordinate
        self.origin_distance = axis_point.pos_from(origin).magnitude()
        self.insertion_distance = axis_point.pos_from(insertion).magnitude()
        self.origin_angle = sm.asin(self.radius / self.origin_distance)
        self.insertion_angle = sm.asin(self.radius / self.insertion_distance)

    @property
    def length(self):
        """Length of the pathway.
        Length of two fixed length line segments and a changing arc length
        of a circle.
        """
        angle = self.origin_angle + self.coordinate + self.insertion_angle
        arc_length = self.radius * angle
        origin_segment_length = self.origin_distance * sm.cos(self.origin_angle)
        insertion_segment_length = self.insertion_distance * sm.cos(
            self.insertion_angle
        )
        return origin_segment_length + arc_length + insertion_segment_length

    @property
    def extension_velocity(self):
        """Extension velocity of the pathway.
        Arc length of circle is the only thing that changes when the elbow
        flexes and extends.
        """
        return self.radius * self.coordinate.diff(me.dynamicsymbols._t)

    def to_loads(self, force_magnitude):
        """Loads in the correct format to be supplied to `KanesMethod`.
        Forces applied to origin, insertion, and P from the muscle wrapped
        over circular arc of radius r.
        """
        self.parent_tangency_point = me.Point("Aw")  # fixed in parent
        self.child_tangency_point = me.Point("Bw")  # fixed in child
        self.parent_tangency_point.set_pos(
            self.axis_point,
            -self.radius * sm.cos(self.origin_angle) * self.parent_axis.cross(self.axis)
            + self.radius * sm.sin(self.origin_angle) * self.parent_axis,
        )
        self.child_tangency_point.set_pos(
            self.axis_point,
            self.radius
            * sm.cos(self.insertion_angle)
            * self.child_axis.cross(self.axis)
            + self.radius * sm.sin(self.insertion_angle) * self.child_axis,
        ),
        parent_force_direction_vector = self.origin.pos_from(self.parent_tangency_point)
        child_force_direction_vector = self.insertion.pos_from(self.child_tangency_point)
        force_on_parent = force_magnitude * parent_force_direction_vector.normalize()
        force_on_child = force_magnitude * child_force_direction_vector.normalize()
        loads = [
            me.Force(self.origin, force_on_parent),
            me.Force(self.axis_point, -(force_on_parent + force_on_child)),
            me.Force(self.insertion, force_on_child),
        ]
        return loads

In [47]:
knee_extensor_pathway = ExtensorPathway(P9, P5, P4, C.z, -C.x, D.x, rk, q4)
knee_extensor_act = bm.FirstOrderActivationDeGroote2016.with_defaults("knee_extensor")
knee_extensor_mus = bm.MusculotendonDeGroote2016.with_defaults(
    "knee_extensor", knee_extensor_pathway, knee_extensor_act
)
knee_flexor_pathway = me.LinearPathway(P9, P5)
knee_flexor_act = bm.FirstOrderActivationDeGroote2016.with_defaults("knee_flexor")
knee_flexor_mus = bm.MusculotendonDeGroote2016.with_defaults(
    "knee_flexor", knee_flexor_pathway, knee_flexor_act
)
ankle_flexor_pathway = me.LinearPathway(P8, P2)
ankle_flexor_act = bm.FirstOrderActivationDeGroote2016.with_defaults("ankle_flexor")
ankle_flexor_mus = bm.MusculotendonDeGroote2016.with_defaults(
    "ankle_flexor", ankle_flexor_pathway, ankle_flexor_act
)
ankle_extensor_pathway = me.LinearPathway(P8, P7)
ankle_extensor_act = bm.FirstOrderActivationDeGroote2016.with_defaults("ankle_extensor")
ankle_extensor_mus = bm.MusculotendonDeGroote2016.with_defaults(
    "ankle_extensor", ankle_extensor_pathway, ankle_extensor_act
)
ankle_damping_B = me.Torque(B, 20.0*u3*B.z)
ankle_damping_C = me.Torque(C, -20.0*u3*B.z)
loads = (
    knee_extensor_mus.to_loads() +
    knee_flexor_mus.to_loads() +
    ankle_flexor_mus.to_loads() +
    ankle_extensor_mus.to_loads() +
    [ankle_damping_B, ankle_damping_C, resistance, gravB, gravC, gravD]
)

# Form the Equations of Motion

In [48]:
kane = me.KanesMethod(
    N,
    (q1, q2),
    (u1, u2),
    kd_eqs=kindiff[:],
    q_dependent=(q3, q4),
    configuration_constraints=holonomic,
    velocity_constraints=mocon,
    u_dependent=(u3, u4),
    constraint_solver="CRAMER",
)
bodies = (crank, foot, lower_leg, upper_leg)
Fr, Frs = kane.kanes_equations(bodies, loads)

muscle_diff_eq = sm.Matrix(
    [
        knee_extensor_mus.a.diff() - knee_extensor_mus.rhs()[0, 0],
        knee_flexor_mus.a.diff() - knee_flexor_mus.rhs()[0, 0],
        ankle_flexor_mus.a.diff() - ankle_flexor_mus.rhs()[0, 0],
        ankle_extensor_mus.a.diff() - ankle_extensor_mus.rhs()[0, 0],
    ]
)

eom = kindiff.col_join(Fr + Frs).col_join(muscle_diff_eq).col_join(holonomic)

In [49]:
state_vars = (
    q1,
    q2,
    q3,
    q4,
    u1,
    u2,
    u3,
    u4,
    knee_extensor_mus.a,
    knee_flexor_mus.a,
    ankle_flexor_mus.a,
    ankle_extensor_mus.a,
)
state_vars

(q₁, q₂, q₃, q₄, u₁, u₂, u₃, u₄, aₖₙₑₑ ₑₓₜₑₙₛₒᵣ, a_knee_flexor, a_ankle_flexor ↪

↪ , aₐₙₖₗₑ ₑₓₜₑₙₛₒᵣ)

# Objective

The objective is to cycle as fast as possible, so we need to find the minmal
time duration to travel a fixed distance $d = r_wGq_1$. This
can be written mathmatically as:
$$
   \min \int^{t_0}_{t_f} dt
$$
For $N$ discretization nodes, this discretizes to:
$$
   \min \sum_1^N h
$$
If $h$ is constant, then we can simply minimize $h$. With the objective being $h$, the gradient of the objective with respect to all of the free optimization variables is zero except for the single entry of
$\frac{\partial h}{\partial h} = 1$.

In [50]:
def obj(free):
    """Return h (always the last element in the free variables)."""
    return free[-1]


def gradient(free):
    """Return the gradient of the objective."""
    grad = np.zeros_like(free)
    grad[-1] = 1.0
    return grad

# Define Numerical Constants



In [51]:
par_map = {
    Ar: 0.55,  # m^2, Tab 5.1, pg 188 Wilson 2004, Upright commuting bike
    CD: 1.15,  # unitless, Tab 5.1, pg 188 Wilson 2004, Upright commuting bike
    Cr: 0.006,  # unitless, Tab 5.1, pg 188 Wilson 2004, Upright commuting bike
    G: 2.0,
    IAzz: 0.0,
    IBzz: 0.01,  # guess, TODO
    ICzz: 0.101,  # lower_leg_inertia [kg*m^2]
    IDzz: 0.282,  # upper_leg_inertia [kg*m^2],
    J: 0.1524,  # from Browser Jason's thesis (rear wheel)
    g: 9.81,
    lam: np.deg2rad(75.0),
    lc: 0.175,  # crank length [m]
    lf: 0.08,  # pedal to ankle [m]
    ll: 0.611,  # lower_leg_length [m]
    ls: 0.8,  # seat tube length [m]
    lu: 0.424,  # upper_leg_length [m],
    m: 85.0,  # kg
    # mA: 0.0,  # not in eom
    mB: 1.0,  # foot mass [kg] guess TODO
    mC: 6.769,  # lower_leg_mass [kg]
    mD: 17.01,  # upper_leg_mass [kg],
    rho: 1.204,  # kg/m^3, air density
    rk: 0.04,  # m, knee radius
    rw: 0.3,  # m, wheel radius
    ankle_extensor_mus.F_M_max: 1000.0,
    ankle_extensor_mus.l_M_opt: np.nan,
    ankle_extensor_mus.l_T_slack: np.nan,
    ankle_flexor_mus.F_M_max: 400.0,
    ankle_flexor_mus.l_M_opt: np.nan,
    ankle_flexor_mus.l_T_slack: np.nan,
    knee_flexor_mus.F_M_max: 1200.0,
    knee_flexor_mus.l_M_opt: np.nan,
    knee_flexor_mus.l_T_slack: np.nan,
    knee_extensor_mus.F_M_max: 1400.0,
    knee_extensor_mus.l_M_opt: np.nan,
    knee_extensor_mus.l_T_slack: np.nan,
}

p = np.array(list(par_map.keys()))
p_vals = np.array(list(par_map.values()))

In [52]:
# To get estimates of the tendon slack length, align the crank with the seat
# tube to maximally extend the knee and hold the foot perpendicular to the
# lower leg then calculate the muscle pathway lengths in this configuration.

q1_ext = -par_map[lam]  # aligned with seat post
q2_ext = 3 * np.pi / 2  # foot perpendicular to crank
eval_holonomic = sm.lambdify((q, p), holonomic)
q3_ext, q4_ext = fsolve(
    lambda x: eval_holonomic([q1_ext, q2_ext, x[0], x[1]], p_vals).squeeze(),
    x0=np.deg2rad([-100.0, 20.0]),
)
q_ext = np.array([q1_ext, q2_ext, q3_ext, q4_ext])

# TODO : duplicated below
eval_ankle_flexor_len = sm.lambdify((q, p), ankle_flexor_pathway.length)
eval_ankle_extensor_len = sm.lambdify((q, p), ankle_extensor_pathway.length)
eval_knee_extensor_len = sm.lambdify((q, p), knee_extensor_pathway.length)
eval_knee_flexor_len = sm.lambdify((q, p), knee_flexor_pathway.length)
# length of muscle path when fully extended
par_map[ankle_flexor_mus.l_T_slack] = eval_ankle_flexor_len(q_ext, p_vals) / 2
par_map[ankle_extensor_mus.l_T_slack] = eval_ankle_extensor_len(q_ext, p_vals) / 2
par_map[knee_extensor_mus.l_T_slack] = eval_knee_extensor_len(q_ext, p_vals) / 2
par_map[knee_flexor_mus.l_T_slack] = eval_knee_flexor_len(q_ext, p_vals) / 2
par_map[ankle_flexor_mus.l_M_opt] = par_map[ankle_flexor_mus.l_T_slack] + 0.01
par_map[ankle_extensor_mus.l_M_opt] = par_map[ankle_extensor_mus.l_T_slack] + 0.01
par_map[knee_extensor_mus.l_M_opt] = par_map[knee_extensor_mus.l_T_slack] + 0.01
par_map[knee_flexor_mus.l_M_opt] = par_map[knee_flexor_mus.l_T_slack] + 0.01

p_vals = np.array(list(par_map.values()))

par_map

{Ar: 0.55, CD: 1.15, Cr: 0.006, F_M_max_ankle_extensor: 1000.0, F_M_max_ankle_ ↪

↪ flexor: 400.0, F_M_max_knee_extensor: 1400.0, F_M_max_knee_flexor: 1200.0, G ↪

↪ : 2.0, IAzz: 0.0, IBzz: 0.01, ICzz: 0.101, IDzz: 0.282, J: 0.1524, g: 9.81,  ↪

↪ l_M_opt_ankle_extensor: 0.07085879260907571, l_M_opt_ankle_flexor: 0.0626301 ↪

↪ 7725283049, l_M_opt_knee_extensor: 0.281592761434192, l_M_opt_knee_flexor: 0 ↪

↪ .25447058267567113, l_T_slack_ankle_extensor: 0.06085879260907571, l_T_slack ↪

↪ _ankle_flexor: 0.052630177252830486, l_T_slack_knee_extensor: 0.271592761434 ↪

↪ 192, l_T_slack_knee_flexor: 0.24447058267567112, lam: 1.3089969389957472, lc ↪

↪ : 0.175, lf: 0.08, ll: 0.611, ls: 0.8, lu: 0.424, m: 85.0, mB: 1.0, mC: 6.76 ↪

↪ 9, mD: 17.01, ρ: 1.204, rk: 0.04, rw: 0.3}

# Muscle Budget and Gear Ratio

Your goal to win the race will be to budget the muscle force production and to select the gear ratio to win the race. That means you can adjust these five parameters:

In [53]:
par_map[G] = 2.0
par_map[ankle_extensor_mus.F_M_max] = 800.0  # [N]
par_map[ankle_flexor_mus.F_M_max] = 1200.0  # [N]
par_map[knee_extensor_mus.F_M_max] = 1000.0  # [N]
par_map[knee_flexor_mus.F_M_max] = 1000.0  # [N]

You have a total budget of 4000 Newtons you can distribute among the four muscles and the gear ratio can be any value greater than zero.

In [54]:
p_vals = np.array(list(par_map.values()))

# Instance Constraints

The cyclist should start with no motion (stationary) and in an initial configuration with the crank forward and horizontal (q1=0 deg) and the toe forward and foot parallel to the crank (q2=180 deg).

In [55]:
q1_0 = 0.0
q2_0 = np.pi
eval_holonomic = sm.lambdify((q, p), holonomic)
q3_0, q4_0 = fsolve(
    lambda x: eval_holonomic([q1_0, q2_0, x[0], x[1]], p_vals).squeeze(),
    x0=np.deg2rad([-90.0, 90.0]),
    xtol=1e-14,
)
q_0 = np.array([q1_0, q2_0, q3_0, q4_0])

# Crank revolutions are proportional to distance traveled so the race distance
# is defined by number of crank revolutions.
distance = 10.0  # m
crank_revs = distance/par_map[rw]/par_map[G]/2.0/np.pi
samples_per_rev = 100
num_nodes = int(crank_revs * samples_per_rev) + 1

h = sm.symbols("h", real=True)

instance_constraints = (
    # set the initial configuration
    q1.replace(t, 0 * h) - q1_0,
    q2.replace(t, 0 * h) - q2_0,
    q3.replace(t, 0 * h) - q3_0,
    q4.replace(t, 0 * h) - q4_0,
    u1.replace(t, 0 * h),  # start stationary
    u2.replace(t, 0 * h),  # start stationary
    u3.replace(t, 0 * h),  # start stationary
    u4.replace(t, 0 * h),  # start stationary
    knee_extensor_mus.a.replace(t, 0 * h),
    knee_flexor_mus.a.replace(t, 0 * h),
    ankle_flexor_mus.a.replace(t, 0 * h),
    ankle_extensor_mus.a.replace(t, 0 * h),
    # at final time travel number of revolutions
    q1.replace(t, (num_nodes - 1) * h) + crank_revs * 2 * np.pi,
)

Only allow forward pedaling and limit the joint angle range of motion. All muscle excitations should be bound between 0 and 1.

In [56]:
bounds = {
    q1: (-(crank_revs + 2) * 2 * np.pi, 0.0),  # can only pedal forward
    # ankle angle, q3=-105 deg: ankle maximally plantar flexed, q3=-30 deg:
    # ankle maximally dorsiflexed
    q3: (-np.deg2rad(105.0), -np.deg2rad(30.0)),
    # knee angle, q4 = 0: upper and lower leg aligned, q4 = pi/2: knee is
    # flexed 90 degs
    q4: (0.0, 3 * np.pi / 2),
    u1: (-30.0, 0.0),  # about 300 rpm
    ankle_extensor_mus.e: (0.0, 1.0),
    ankle_flexor_mus.e: (0.0, 1.0),
    knee_flexor_mus.e: (0.0, 1.0),
    knee_extensor_mus.e: (0.0, 1.0),
    h: (0.0, 0.1),
}

# Instantiate the Optimal Control Problem

In [ ]:
problem = Problem(
    obj,
    gradient,
    eom,
    state_vars,
    num_nodes,
    h,
    known_parameter_map=par_map,
    instance_constraints=instance_constraints,
    bounds=bounds,
)
problem.add_option("nlp_scaling_method", "gradient-based")
problem.add_option("max_iter", 2000)

initial_guess = np.random.normal(loc=0.5, scale=0.2, size=problem.num_free)

q1_guess = np.linspace(0.0, -crank_revs*2*np.pi, num=num_nodes)
q2_guess = np.linspace(0.0, crank_revs*2*np.pi, num=num_nodes)

u1_guess = np.linspace(0.0, -40.0, num=num_nodes)
u1_guess[num_nodes//2:] = -20.0
u2_guess = np.linspace(0.0, 40.0, num=num_nodes)
u2_guess[num_nodes//2:] = 20.0

initial_guess[0*num_nodes:1*num_nodes] = q1_guess
initial_guess[1*num_nodes:2*num_nodes] = q2_guess
initial_guess[4*num_nodes:5*num_nodes] = u1_guess
initial_guess[5*num_nodes:6*num_nodes] = u2_guess
initial_guess[-1] = 0.02

# Plot Initial Guess

In [ ]:
_ = problem.plot_trajectories(initial_guess)

# Solve the Optimal Control Problem

In [ ]:
solution, info = problem.solve(initial_guess)
xs, us, ps, h_val = parse_free(
    solution, len(state_vars), 4, num_nodes, variable_duration=True
)
print("Optimal value h:", solution[-1])
print(info["status_msg"])

In [ ]:
_ = problem.plot_objective_value()

In [ ]:
problem.plot_constraint_violations(solution)

In [ ]:
_ = problem.plot_trajectories(solution)

In [ ]:
eval_mus_forces = sm.lambdify(
    (state_vars, p),
    (ankle_extensor_mus.force.doit().xreplace(qd_repl),
     ankle_flexor_mus.force.doit().xreplace(qd_repl),
     knee_flexor_mus.force.doit().xreplace(qd_repl),
     knee_extensor_mus.force.doit().xreplace(qd_repl)),
    cse=True)

eval_mus_lens = sm.lambdify((q, p),
                            (ankle_extensor_mus.pathway.length.xreplace(qd_repl),
                             ankle_flexor_mus.pathway.length.xreplace(qd_repl),
                             knee_flexor_mus.pathway.length.xreplace(qd_repl),
                             knee_extensor_mus.pathway.length.xreplace(qd_repl)),
                            cse=True)

eval_mus_vels = sm.lambdify((state_vars, p),
                            (ankle_extensor_mus.pathway.extension_velocity.xreplace(qd_repl),
                             ankle_flexor_mus.pathway.extension_velocity.xreplace(qd_repl),
                             knee_flexor_mus.pathway.extension_velocity.xreplace(qd_repl),
                             knee_extensor_mus.pathway.extension_velocity.xreplace(qd_repl)),
                            cse=True)

ake_for, akf_for, knf_for, kne_for = eval_mus_forces(xs, p_vals)
ake_len, akf_len, knf_len, kne_len = eval_mus_lens(xs[:4], p_vals)
ake_vel, akf_vel, knf_vel, kne_vel = eval_mus_vels(xs, p_vals)


def plot_sim_compact():

    time = np.linspace(0, num_nodes * h_val, num=num_nodes)

    fig, axes = plt.subplots(5, 1, sharex=True, layout="constrained", figsize=(10.0, 10.0))

    axes[0].set_title("Finish time = {:1.3f}".format(time[-1]))
    axes[0].plot(time, -ake_for, time, -akf_for, time, -knf_for, time, -kne_for)
    axes[0].set_ylabel("Force\n[N]")
    # axes[0].legend(['Ankle Extensor', 'Ankle Flexor', 'Knee Flexor', 'Knee Extensor'])

    axes[1].plot(time, ake_len, time, akf_len, time, knf_len, time, kne_len)
    axes[1].legend(["Ankle Extensor", "Ankle Flexor", "Knee Flexor", "Knee Extensor"])
    axes[1].set_ylabel("Length\n[M]")

    axes[2].plot(time, -xs[4, :] * 60 / 2 / np.pi, time, xs[5, :] * 60 / 2 / np.pi)
    axes[2].set_ylabel("Cadence\n[RPM]")
    axes[2].legend(["Cadence", "Pedal Cadence"])

    axes[3].plot(time, us[0:2, :].T)
    axes[3].legend(problem.collocator.unknown_input_trajectories[0:2])
    axes[3].set_ylabel("Excitation")

    axes[4].plot(time, us[2:4, :].T)
    axes[4].legend(problem.collocator.unknown_input_trajectories[2:4])
    axes[4].set_ylabel("Excitation")

    axes[-1].set_xlabel("Time [s]")

    return axes


_ = plot_sim_compact()

# Plot Bike Speed and Rider Power

In [ ]:
kin_pow = (2*J + m*rw**2)*G**2*u1.diff()*u1
roll_pow = -Cr*m*g*rw*G*u1
air_pow = -rho*CD*Ar*G**3*rw**3*u1**2/2*u1
eval_pow = sm.lambdify((u1.diff(t), u1, p),
                       (kin_pow, roll_pow, air_pow))
time = np.linspace(0, num_nodes*h_val, num=num_nodes)
u1ds = np.diff(xs[4, :], prepend=0.0)/np.diff(time, prepend=-h_val)
kps, rps, aps = eval_pow(u1ds, xs[4, :], p_vals)


def plot_speed_power():

    fig, axes = plt.subplots(5, 1, sharex=True, layout='constrained',
                             figsize=(10.0, 10.0))

    axes[0].set_title('Finish time = {:1.3f} s'.format(time[-1]))
    axes[0].plot(time, -xs[4, :]*60/2/np.pi,
                 time, xs[5, :]*60/2/np.pi)
    axes[0].set_ylabel('Cadence\n[RPM]')
    axes[0].legend(['Cadence', 'Pedal Cadence'])

    axes[1].plot(time, -xs[4, :]*par_map[G]*par_map[rw]*3.6)
    axes[1].set_ylabel('Speed [km/h]')

    axes[2].plot(time, rps, time, aps)
    axes[2].set_ylabel('Power [W]')
    axes[2].legend(['Rolling', 'Air'])

    axes[3].plot(time, kps,
                 time, kps + rps + aps)
    axes[3].axhline(np.mean(kps + rps + aps), color='black')
    axes[3].set_ylabel('Power [W]')
    axes[3].legend(['Kinetic', 'Total', 'Average'])

    axes[4].plot(time, ake_for*ake_vel, time, akf_for*akf_vel, time, knf_for*knf_vel, time, kne_for*kne_vel)
    axes[4].plot(time, ake_for*ake_vel + akf_for*akf_vel + knf_for*knf_vel + kne_for*kne_vel)
    axes[4].legend(["Ankle Extensor", "Ankle Flexor", "Knee Flexor", "Knee Extensor", "Total Muscle"])
    axes[4].set_ylabel('Power [W]')

    axes[-1].set_xlabel('Time [s]')

    return axes


_ = plot_speed_power()

# Plot Configuration

In [ ]:
plot_points = [P6, P1, P2, P3, P7, P3, P4, P5]
coordinates = P6.pos_from(P1).to_matrix(N)
for Pi in plot_points[1:]:
    coordinates = coordinates.row_join(Pi.pos_from(P1).to_matrix(N))
eval_coordinates = sm.lambdify((q, p), coordinates)

mus_points = [P7, P8, P2, P8, None, P9, P6,
              knee_extensor_pathway.child_tangency_point, None,
              knee_extensor_pathway.parent_tangency_point, P9]
mus_coordinates = P7.pos_from(P1).to_matrix(N)
for Pi in mus_points[1:]:
    if Pi is None:
        pi_coord = sm.Matrix([sm.nan, sm.nan, sm.nan])
    else:
        pi_coord = Pi.pos_from(P1).to_matrix(N)
    mus_coordinates = mus_coordinates.row_join(pi_coord)
eval_mus_coordinates = sm.lambdify((q, p), mus_coordinates)

title_template = 'Time = {:1.2f} s'


def plot_configuration(q_vals, p_vals, ax=None):
    if ax is None:
        fig, ax = plt.subplots(layout='constrained')

    x, y, _ = eval_coordinates(q_vals, p_vals)
    xm, ym, _ = eval_mus_coordinates(q_vals, p_vals)
    crank_circle = plt.Circle((0.0, 0.0), par_map[lc], fill=False,
                              linestyle='--')
    bike_lines, = ax.plot(x[:3], y[:3], 'o-', linewidth=2, color='#3dcfc2ff')
    leg_lines, = ax.plot(x[2:], y[2:], 'o-', linewidth=4, color='#ffd90fff')
    mus_lines, = ax.plot(xm, ym, 'o-', color='#800080ff',)
    knee_circle = plt.Circle((x[6], y[6]), par_map[rk], color='#800080ff',
                             fill=False)
    ax.add_patch(crank_circle)
    ax.add_patch(knee_circle)
    title_text = ax.set_title(title_template.format(0.0))
    ax.set_aspect('equal')
    return ax, fig, bike_lines, leg_lines, mus_lines, knee_circle, title_text

Fully extended configuration:

In [ ]:
_ = plot_configuration(q_ext, p_vals)

Initial configuration:

In [ ]:
ax, fig, bike_lines, leg_lines, mus_lines, knee_circle, title_text = plot_configuration(q_0, p_vals);

# Animation

In [ ]:
def animate(i):
    qi = xs[0:4, i]
    x, y, _ = eval_coordinates(qi, p_vals)
    xm, ym, _ = eval_mus_coordinates(qi, p_vals)
    bike_lines.set_data(x[:3], y[:3])
    leg_lines.set_data(x[2:], y[2:])
    mus_lines.set_data(xm, ym)
    knee_circle.set_center((x[6], y[6]))
    title_text.set_text('Time = {:1.2f} s'.format(i*h_val))


ani = animation.FuncAnimation(fig, animate, range(0, num_nodes, 2), interval=int(h_val * 4000))
HTML(ani.to_jshtml())